## Market Intellegience Project 1: Product Design

### I. Project Introduction

In this project, we are the consumer insights team for an established, small to medium-sized beverage company. Te company is interested in capitalizing on recent market growth in botled, health-oriented beverages (e.g., flavored/enhanced water, cold teas, freshly pressed juice) and is considering launching a new product in this category. 

Our task is to identify a concept for a new product in the botled, health-oriented beverage category, design a consumer survey that will generate quantitative data, and analyze these data to inform whether the firm should launch this product or not. 

**To finish this market research, we will firstly design a survey on Qualtric, visualize and explore the survey results, conduct conjoint analysis and give the company a final proposal.** 

### II. Survey Design and Data Visualization

To generate quantitative data, we designed a survey containing 12 questions on Qualtric as is shown in Qualtric **Survey.pdf** in this repository.

After getting data, we used R to explore and visualize the survey results. The codes are given in the **Data Visualization.R** file in this repository.

To understand the **target segments** for the new product concept, we firstly did a boxplot to discover the relationship between customer age and their likeliness to buy the AIR Water. As the box plot indicates, the customers under 44 years old have an increased likeliness to buy the infused water as their age increases. However, for customers older than 44 years old, the likeliness to buy the product is decreasing and is lower than any of the younger groups. Thus, the new product should target customers who are **younger than 44**.

To explore the relationship between customers’ willingness to pay for the product and their **education level**, we generated a line plot on Qualtrics. As the plot indicates, people with higher education levels are more likely to offer a higher desired price for purchasing the product. We can also tell from the graph that potential customers who have professional degrees are giving a higher expected price for the product compared with other education level groups. Thus, we came to the conclusion that we should target customers who are **college students or are having a higher education**.

We can conclude from the results we obtained that our target audiences consume high carbonated/caffeinated products, care most about sugar content followed by flavor, price and nutritional value and prefer non-fizzy herbs infused product. The main motivation of respondents is to detoxify their bodies and improve health so we can use these keywords in our product descriptions. Also, on average they are willing to pay $2.4 for a bottle and we can consider it after doing our break-even analysis.

Having said that, around 60% of respondents do not consume this type of product and on average there is a 50% chance that they will buy this product in the future.

To conduct further research on costumers' preference, we build on the survey. We came up with a couple of potential product concepts to test. We will use ratings-based conjoint analysis to determine the beverage atributes that will appeal to the greatest number of people. The survey is given in **Conjoint–Survey.pdf**.


### III.  Conjoint Analysis

#### 1. Study Design

Import the experimental design builder script.

In [ ]:
import sys
sys.path.insert(1, '/data/ProductDesign')
import designer

Define a variable called `attributes` that contains the attributes and levels within each attribute.

In [ ]:
attributes = {"Sugar content": ["With Added Sugar", "No Added Sugar"],
             "Carbonation": ["Carbonated", "Not Carbonated"],
             "Packaging":["Plastic Bottle","Glass Bottle"],
             "Bottle Cap":["Flip Cap","Twist Cap"]}

Use the designer script to find a matching design.

In [ ]:
import pandas as pd
design = pd.DataFrame({"Product":[str(i) for i in range(1,17)],
"Sugar content": ["With Added Sugar", "No Added Sugar"]*8,
"Carbonation": (["Carbonated"]*2 + ["Not Carbonated"]*2)*4,
"Packaging": (["Plastic Bottle"]*4 + ["Glass Bottle"]*4)*2,
"Bottle Cap": (["Flip Cap"]*8+["Twist Cap"]*8)})

#### 2. Write and administer the survey

In [ ]:
#Read the data
import pandas as pd
raw_survey_results =  pd.read_csv('Conjoint.csv')

Clean the data.

In [ ]:
#Drop Unnecesary Columns
remove_columns = raw_survey_results.drop(['StartDate','EndDate','Status','IPAddress','Progress','Duration (in seconds)','Finished','RecordedDate',
                        'RecipientLastName','RecipientFirstName','RecipientEmail','ExternalReference','LocationLatitude','LocationLongitude',
                        'DistributionChannel','UserLanguage'],axis=1)
#Remove Unnecessary rows
remove_rows = remove_columns.drop(remove_columns.index[[0,1,2,3,4,5]])
# Rename Columns
clean = remove_rows.rename(columns={"Q1_1": "9",
                                    "Q1_2": "1",
                                    "Q1_3": "13",
                                    "Q1_4" : "5",
                                    "Q1_5":"11",
                                    "Q1_6": "3",
                                    "Q1_7" : "15",
                                    "Q1_8":"7",
                                    "Q1_9":"10",
                                    "Q1_10":"2",
                                    "Q1_11":"14",
                                    "Q1_12":"6",
                                    "Q1_13":"12",        
                                    "Q1_14":"4",
                                    "Q1_15":"16",
                                    "Q1_16":"8"})
#Melting or Unpivot
almost_there = pd.melt(clean, id_vars=['ResponseId'], value_vars=[str(i) for i in range(1,17)])
melt = almost_there.rename(columns={"variable": "Product"})

import numpy as np

#melt.merge()
melt_new = melt.replace(np.nan, '1', regex=True)
clean_survey_results_at = melt_new.merge(design, on='Product')
clean_survey_results = clean_survey_results_at.drop('Product', axis=1)

Introduce dummy variables in preparation for anlaysis. Now, for each attribute, there should be the number of levels minus one. (The intercept captures the "default" level, and the part-worths for all other levels will be measured relative to the default utility.)

In [ ]:
clean_survey_results['Sugar content_alt'] = clean_survey_results['Sugar content'] == "With Added Sugar"
clean_survey_results['Carbonation_alt'] = clean_survey_results['Carbonation'] == "Carbonated"
clean_survey_results['Packaging_alt'] = clean_survey_results['Packaging'] == "Plastic Bottle"
clean_survey_results['Bottle Cap_alt'] = clean_survey_results['Bottle Cap'] == "Twist Cap"
data = clean_survey_results.drop(['Sugar content','Carbonation','Packaging','Bottle Cap'],axis=1)

#### 4. Estimate the average and individual part-worths
We will use sklearn's implementation of linear regression to estimate part-worths.

In [ ]:
from sklearn.linear_model import LinearRegression

Compute the average part-worths by:

* splitting the data into y (the response/rating) and X (the explanatory variables/attribute levels), and
* Run a linear regression to estimate intercept and coefficients.

In [ ]:
x = data[['Sugar content_alt','Carbonation_alt','Packaging_alt','Bottle Cap_alt']] 
y = data[['value']]
regressor = LinearRegression()
regressor.fit(x, y)
print('Coefficients: \n', regressor.coef_)

Calculate the importance of each attribute by doing the following steps.

* Compute the part-worth range for each attribute.
* Calculate the total range, summing the ranges over each attribute.
* Calculate the importance for a single attribute as the range for that attribute over the total range.

In [ ]:
intercept = float(regressor.intercept_)
coe = regressor.coef_
range= abs(coe - intercept)
total_range = range.sum()
importance = range / total_range 
print('part-worth range:', range) 
print('total range:', total_range)
print('importance:', importance)

According to the part-worths, ranges and importance of each attribute, the ideal product will have **no added sugar, will be carbonated and packaged in a glass bottle which has a twist cap**.

Now calculate **individual part-worths**. We will do this by dividing up the data by individual respondent and running one regression for each respondent.

In [ ]:
Id_list=data['ResponseId'].unique().tolist()
individual = []
for i in Id_list:
    sub = data[data['ResponseId'] == i]
    x = sub[['Sugar content_alt','Carbonation_alt','Packaging_alt','Bottle Cap_alt']] 
    y = sub[['value']]
    regressor = LinearRegression()
    regressor.fit(x, y)
    individual_coe = regressor.coef_
    intercept = float(regressor.intercept_)
    keys = ['ID','Intercept','sugar_coe','carbonation_coe','packaging_coe','cap_coe']
    values = [i,intercept,individual_coe[0][0],individual_coe[0][1],individual_coe[0][2],individual_coe[0][3]]
    individual.append(dict(zip(keys, values)))
individual_part_worths = pd.DataFrame(individual)
individual_part_worths

#### 5. Simulate the market

Write a function utility that takes two arguments:

* product: a list of which level has been selected for each attribute. 
* part_worths: a row from the individual_part_worths data frame created.

The utility function will compute and return the utility of the specified individual for the specified product; this is the intercept plus any relevant part-worths that correspond to the product's attribute levels.

In [ ]:
def utility(product,part_worths):
    u = part_worths.iloc[0]["Intercept"]
    if product[0] == "With Added Sugar":
        u = u + part_worths.iloc[0]["sugar_coe"]
    if product[1] == "Carbonated":
        u = u + part_worths.iloc[0]["carbonation_coe"]
    if product[2] == "Plastic Bottle":
        u = u + part_worths.iloc[0]["packaging_coe"]
    if product[3] == "Twist Cap":
        u = u + part_worths.iloc[0]["cap_coe"]
    return u

#test the function
product = ["With Added Sugar","Carbonated","PLastic Bottle","Twist Cap"]
part_worths = individual_part_worths.loc[individual_part_worths.ID=="R_3HqaVcuLFhFcFE6"]
utility(product,part_worths)

Write a function choose_product that takes two arguments:

* products: a list of products, where each product is its own list as specified before
* part_worths: a row of inidivudual part worths, just as used in Task 8.

This fuction will return a numpy array with the same length as the number of products; this array will contain a boolean value (true or false) for each product. The value for a product should be true if it maximizes the utility for the individual; as multiple products can have equivalent utilities, more than one product might maximize utility.

In [ ]:
def choose_product(products,part_worths):
    us = []
    ulist = []
    for product in products:  
        u = utility(product,part_worths)
        us.append(u)
    umax = max(us)
    for each in us:
        if each == umax:
            ulist.append(True)
        else:
            ulist.append(False)
    ulist=np.array(ulist)
    return ulist

#test the function
products = [["With Added Sugar","Carbonated","PLastic Bottle","Twist Cap"],["No Added Sugar","Carbonated","PLastic Bottle","Twist Cap"]]
part_worths = individual_part_worths.loc[individual_part_worths.ID=="R_3HqaVcuLFhFcFE6"]
choose_product(products,part_worths)

Write a function simulate that takes two arguments:

* products: a list of products, identitcal to the argument for Task 9.
* The individual_part_worths data frame created for Task 7.

This function will simulate the market share of the products by having each individual choose a product from the list. For an individual who chooses more than one item (multiple items are tied for the top utility value), their "vote" is split between the tied items. The function will return an array of proportions of market share.

In [ ]:
Id_list=data['ResponseId'].unique().tolist()
def simulate(products, individual_part_worths):
    votes=[]
    for i in Id_list:
        part_worths = individual_part_worths.loc[individual_part_worths.ID==i]
        vote = choose_product(products,part_worths)
        votes.append(vote)
    table = pd.DataFrame(list(map(np.ravel, votes)))
    table.replace(False, 0, inplace=True)
    table.replace(True, 1, inplace=True)
    sum = table.sum(axis=0)
    total = sum[0]+sum[1]+sum[2]+sum[3]
    share = [sum[0]/total,sum[1]/total,sum[2]/total,sum[3]/total]
    share = np.array(share)
    return share

Create a variable called my_product that has the ideal value (maximum average utility) based on what we discovered; Also create three competitiors using this same format. Create a list called all_products that contains all four products and simulate the market with the products.

For competitor products, we have identified the following products as competitors:
- A bottle of water with no sugar, no carbonation in a plastic bottle with a twist cap
- A bottle of water with no sugar, no carbonation in a glass bottle with a twist cap
- A bottle of water with no sugar, carbonation in a plastic bottle with a flip cap

We picked these products to simulate the existing products on the market such as bottled waters like Dasani, Acqua Panna, Mineral Springs. This will give us an idea on how our product will perform in the market compared to existing products.

In [ ]:
my_product = ["No Added Sugar","Carbonated","Glass Bottle","Twist Cap"]
competitor1 = ["No Added Sugar","Not Carbonated","Plastic Bottle","Twist Cap"]
competitor2 = ["No Added Sugar","Not Carbonated","Glass Bottle","Twist Cap"]
competitor3 = ["No Added Sugar","Carbonated","Plastic Bottle","Flip Cap"]
all_products = [my_product,competitor1,competitor2,competitor3]
simulate(all_products,individual_part_worths)

The forecasted market share for our product is 45.45%.

Thus, we suggest the company to launch the ideal product.